# Composing Contracts II

Toying with ideas from [How to Write a Financial Contract](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.14.7885) by S. L. Peyton Jones and J-M. Eber. There is an old version of the paper [Composing Contracts:An Adventure in Financial Engineering](https://www.cs.tufts.edu/~nr/cs257/archive/simon-peyton-jones/contracts.pdf) by Jones, Eber and Seward. Some code for this version can be found in [Composing Contracts](./Composing%20Contracts.ipynb).

In [1]:
from typing import TypeVar, Generic
from dataclasses import dataclass, asdict, astuple
from abc import ABC, abstractmethod
from datetime import datetime

## Observable Floats and Visitor ABC

In [2]:
class ObservableFloat(ABC):
    pass

In [3]:
@dataclass
class Konst(ObservableFloat):
    constant: float

@dataclass
class Stock(ObservableFloat):
    ticker: str

In [4]:
T = TypeVar('T')

class ObservableFloatVisitor(ABC, Generic[T]):
    
    @abstractmethod
    def konst(self, constant: float) -> T: pass

    @abstractmethod
    def stock(self, ticker: str) -> T: pass
    
    def __call__(self, observable: ObservableFloat) -> T:
        if isinstance(observable, Konst):
            return self.konst(observable.constant)
        elif isinstance(observable, Stock):
            return self.stock(observable.ticker)
        else:
            raise TypeError(f'Unknown observable float type "{type(contract).__name__}"')

## Observable Booleans and Visitor ABC

In [5]:
class ObservableBool(ABC):
    pass

In [6]:
@dataclass
class KonstBool(ObservableBool):
    constant: bool

@dataclass
class At(ObservableBool):
    date: datetime

In [7]:
T = TypeVar('T')

class ObservableBoolVisitor(ABC, Generic[T]):
    
    @abstractmethod
    def konst(self, constant: bool) -> T: pass

    @abstractmethod
    def at(self, date: datetime) -> T: pass
    
    def __call__(self, observable: ObservableBool) -> T:
        if isinstance(observable, KonstBool):
            return self.konst(observable.constant)
        elif isinstance(observable, At):
            return self.at(observable.date)
        else:
            raise TypeError(f'Unknown observable bool type "{type(contract).__name__}"')

## Contracts and Visitor ABC

In [8]:
class Contract(ABC):
    pass

In [9]:
@dataclass
class Zero(Contract):
    pass

@dataclass
class One(Contract):
    currency: str

@dataclass
class Give(Contract):
    contract: Contract

@dataclass
class And(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Or(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Cond(Contract):
    observable: ObservableBool
    contract1: Contract
    contract2: Contract    

@dataclass
class Scale(Contract):
    observable: ObservableFloat
    contract: Contract

@dataclass
class When(Contract):
    observable: ObservableBool
    contract: Contract

@dataclass
class Anytime(Contract):
    observable: ObservableBool
    contract: Contract

@dataclass
class Until(Contract):
    observable: ObservableBool
    contract: Contract

In [10]:
T = TypeVar('T')

class ContractVisitor(ABC, Generic[T]):

    @abstractmethod
    def zero(self) -> T: pass

    @abstractmethod
    def one(self, currency: str) -> T: pass

    @abstractmethod
    def give(self, contract: Contract) -> T: pass

    @abstractmethod
    def and_(self, contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def or_(self, contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def cond(self, observable: ObservableBool, contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def scale(self, observable: ObservableFloat, contract: Contract) -> T: pass

    @abstractmethod
    def when(self, observable: ObservableBool, contract: Contract) -> T: pass

    @abstractmethod
    def anytime(self, observable: ObservableBool, contract: Contract) -> T: pass

    @abstractmethod
    def until(self, observable: ObservableBool, contract: Contract) -> T: pass

    def __call__(self, contract: Contract) -> T:
        if isinstance(contract, Zero):
            return self.zero()
        elif isinstance(contract, One):
            return self.one(**contract.__dict__)
        elif isinstance(contract, Give):
            return self.give(**contract.__dict__)
        elif isinstance(contract, And):
            return self.and_(**contract.__dict__)
        elif isinstance(contract, Or):
            return self.or_(**contract.__dict__)
        elif isinstance(contract, Cond):
            return self.cond(**contract.__dict__)
        elif isinstance(contract, Scale):
            return self.scale(**contract.__dict__)
        elif isinstance(contract, When):
            return self.when(**contract.__dict__)
        elif isinstance(contract, Anytime):
            return self.anytime(**contract.__dict__)
        elif isinstance(contract, Until):
            return self.until(**contract.__dict__)
        else:
            raise TypeError(f'Unknown contract type "{type(contract).__name__}"')

## A Simple Printer Implementation of Visitors

In [11]:
class ObservableFloatPrinter(ObservableFloatVisitor[str]):
    def konst(self, constant: float) -> str: return str(constant)
    def stock(self, ticker: str) -> str: return f'"{ticker}"'

In [12]:
class ObservableBoolPrinter(ObservableBoolVisitor[str]):
    def konst(self, constant: bool) -> str: return str(constant)
    def at(self, date: datetime) -> str: return f'"{date}"'

In [13]:
class ContractPrinter(ContractVisitor[str]):
    
    def __init__(self,
                 observable_float_visitor: ObservableFloatVisitor[str],
                 observable_bool_visitor: ObservableBoolVisitor[str]) -> None:
        self.observable_float_visitor = observable_float_visitor
        self.observable_bool_visitor = observable_bool_visitor
    
    def zero(self) -> str: return 'zero'
    def one(self, currency: str) -> str: return f'one {currency}'
    def give(self, contract: Contract) -> str: return f'give ({self(contract)})'
    def and_(self, contract1: Contract, contract2: Contract) -> str: return f'and ({self(contract1)}) ({self(contract2)})'
    def or_(self, contract1: Contract, contract2: Contract) -> str: return f'or ({self(contract1)}) ({self(contract2)})'
    def cond(self, observable: ObservableBool, contract1: Contract, contract2: Contract) -> T: return f'cond ({self.observable_bool_visitor(observable)}) ({self(contract1)}) ({self(contract2)})'
    def scale(self, observable: ObservableFloat, contract: Contract) -> str: return f'scale {self.observable_float_visitor(observable)} ({self(contract)})'
    def when(self, observable: ObservableBool, contract: Contract) -> T: return f'when {self.observable_bool_visitor(observable)} ({self(contract)})'
    def anytime(self, observable: ObservableBool, contract: Contract) -> T: return f'anytime {self.observable_bool_visitor(observable)} ({self(contract)})'
    def until(self, observable: ObservableBool, contract: Contract) -> T: return f'until {self.observable_bool_visitor(observable)} ({self(contract)})'


## Examples

In [14]:
def zcb(maturity: datetime, notional: float, currency: str):
    return When(At(maturity), Scale(Konst(notional), One(currency)))

In [15]:
def european(maturity: datetime, contract: Contract):
    return When(At(maturity), Or(contract, Zero()))

In [16]:
mybond1 = zcb(datetime(2030, 7, 14), 1000000, 'EUR')
mybond1

When(observable=At(date=datetime.datetime(2030, 7, 14, 0, 0)), contract=Scale(observable=Konst(constant=1000000), contract=One(currency='EUR')))

In [17]:
ContractPrinter(ObservableFloatPrinter(), ObservableBoolPrinter())(mybond1)

'when "2030-07-14 00:00:00" (scale 1000000 (one EUR))'

In [18]:
asdict(mybond1)

{'observable': {'date': datetime.datetime(2030, 7, 14, 0, 0)},
 'contract': {'observable': {'constant': 1000000},
  'contract': {'currency': 'EUR'}}}

In [19]:
astuple(mybond1)

((datetime.datetime(2030, 7, 14, 0, 0),), ((1000000,), ('EUR',)))

In [20]:
dict(**mybond1.__dict__)

{'observable': At(date=datetime.datetime(2030, 7, 14, 0, 0)),
 'contract': Scale(observable=Konst(constant=1000000), contract=One(currency='EUR'))}